## Log file transformation playground

**Note**

A client connection is established when both Association and Authoriation are successful

How can a unique client be indentified? IP or Mac?
- https://community.nxp.com/t5/Wireless-Connectivity-Knowledge/802-11-Wi-Fi-Connection-Disconnection-process/ta-p/1121148 

Mac Address is better for identifying devices (IPs can change)
- https://www.cleverfiles.com/howto/what-is-mac-address.html

So in order for a connection to be established, both authorization and association need to succeed. When parsing the logs, need to check for Both auth and assoc in sequence.

Lots of duplicates in the data as well - mostly on assoc requests

Access Points also have mac addresses...

In [1]:
from IPython.display import IFrame
IFrame("Uncc_Campus_Temporal_Map24x36.pdf", width=900, height=800)

In [2]:
import os
import pandas as pd

In [3]:
os.getcwd()

'/home/calvinhathcock/Documents/College/UNCC/Fall 2022/ITSC 4155/Project/ETL'

In [4]:
case = []
path = '/media/calvinhathcock/Secondary SSD/ITSC_4155_Capstone/dataset/raw/wifi/var/log/remote/wireless-encoded/wireless_09-29-2020.log'
with open(path, 'r') as file:
    for line in file:
        if '38:f9:d3:2e:47:62' in line:
            case.append(line)
            print(line)
    

Sep 29 11:24:01 10.44.80.228 stm[2528]:  <501093> <NOTI> |AP EXT-Levi184-AP275-1@10.44.80.228 stm|  Auth success: 38:f9:d3:2e:47:62: AP 10.44.80.228-b4:5d:50:13:45:61-EXT-Levi184-AP275-1

Sep 29 11:24:01 10.44.80.228 stm[2528]:  <501095> <NOTI> |AP EXT-Levi184-AP275-1@10.44.80.228 stm|  Assoc request @ 11:24:01.468846: 38:f9:d3:2e:47:62 (SN 1559): AP 10.44.80.228-b4:5d:50:13:45:61-EXT-Levi184-AP275-1

Sep 29 11:24:01 10.47.0.32 stm[1918]: <501218> <1918> <NOTI> < 10.47.0.32>  stm_sta_assign_vlan 21120: VLAN: sta 38:f9:d3:2e:47:62, STM assigns MAC based vlan_id 1

Sep 29 11:24:01 10.47.0.32 stm[1918]: <501100> <1918> <NOTI> < 10.47.0.32>  Assoc success @ 11:24:01.479592: 38:f9:d3:2e:47:62: AP 10.44.80.228-b4:5d:50:13:45:61-EXT-Levi184-AP275-1

Sep 29 11:24:01 10.44.80.228 stm[2528]:  <501100> <NOTI> |AP EXT-Levi184-AP275-1@10.44.80.228 stm|  Assoc success @ 11:24:01.475780: 38:f9:d3:2e:47:62: AP 10.44.80.228-b4:5d:50:13:45:61-EXT-Levi184-AP275-1

Sep 29 11:24:20 10.44.80.205 stm[3548]: 

Sep 29 13:27:36 10.45.65.170 stm[3808]:  <501093> <NOTI> |AP Atki300-AP345-3@10.45.65.170 stm|  Auth success: 38:f9:d3:2e:47:62: AP 10.45.65.170-48:4a:e9:bd:17:11-Atki300-AP345-3

Sep 29 13:27:36 10.47.0.32 stm[1918]: <501218> <1918> <NOTI> < 10.47.0.32>  stm_sta_assign_vlan 21120: VLAN: sta 38:f9:d3:2e:47:62, STM assigns MAC based vlan_id 1

Sep 29 13:27:36 10.47.0.32 stm[1918]: <501100> <1918> <NOTI> < 10.47.0.32>  Assoc success @ 13:27:36.241954: 38:f9:d3:2e:47:62: AP 10.45.65.170-48:4a:e9:bd:17:11-Atki300-AP345-3

Sep 29 13:27:36 10.45.65.170 stm[3808]:  <501095> <NOTI> |AP Atki300-AP345-3@10.45.65.170 stm|  Assoc request @ 13:27:36.240783: 38:f9:d3:2e:47:62 (SN 2299): AP 10.45.65.170-48:4a:e9:bd:17:11-Atki300-AP345-3

Sep 29 13:27:36 10.45.65.170 stm[3808]:  <501100> <NOTI> |AP Atki300-AP345-3@10.45.65.170 stm|  Assoc success @ 13:27:36.244195: 38:f9:d3:2e:47:62: AP 10.45.65.170-48:4a:e9:bd:17:11-Atki300-AP345-3

Sep 29 13:27:36 Aruba-MM-01 wms[6292]: <126038> <6292> <WARN> <Aruba

Oct  3 00:00:00
Sep 29 00:00:00
15 spaces

In [5]:
#trim time stamp
case[0][:15]

'Sep 29 11:24:01'

For a given mac address, find all full connections (assoc success AND auth success in sequence)

Assoc Request: <501095>
Assoc Success: <501100>
Auth Success: <501093>
Deauth1 (roaming?): <501080>
Deauth2: <501105>
Deauth3: <501106>

There may be more deauth codes will check later

In [6]:
import warnings
warnings.filterwarnings('ignore')

def analyze_mac_address(macs):
    ''' not suitable for many mac addresses - not optimized'''
    def count_logs(mac_address):
        case = []
        path = '/media/calvinhathcock/Secondary SSD/ITSC_4155_Capstone/dataset/raw/wifi/var/log/remote/wireless-encoded/wireless_09-29-2020.log'

        assoc_req = 0
        assoc_succ = 0
        auths = 0
        deauth1 = 0
        deauth2 = 0
        deauth3 = 0

        with open(path, 'r') as file:
            for line in file:
                if mac_address in line:
                    case.append(line)

        for c in case:
            if '<501095>' in c:
                assoc_req += 1
            if '<501100>' in c:
                assoc_succ += 1
            if '<501093>' in c:
                auths += 1
            if '<501080>' in c:
                deauth1 += 1
            if '<501105>' in c:
                deauth2 += 1
            if '<501106>' in c:
                deauth3 += 1

        return assoc_req, assoc_succ, auths, deauth1, deauth2, deauth3
    
    mac_info_df = pd.DataFrame(columns = ['mac_address', 'assoc_req_count', 'assoc_succ_count', 'auths_count', 'deauth1_count', 'deauth2_count', 'deauth3_count'])

    for m in macs:
        assoc_req, assoc_succ, auths, deauth1, deauth2, deauth3 = count_logs(m)

        data = {
            'mac_address': m,
            'assoc_req_count': assoc_req,
            'assoc_succ_count': assoc_succ,
            'auths_count': auths,
            'deauth1_count': deauth1, 
            'deauth2_count': deauth2, 
            'deauth3_count': deauth3
        }

        mac_info_df = mac_info_df.append(data, ignore_index=True)
    
    
    mac_info_df['assoc_succ_count'] = mac_info_df['assoc_succ_count'] / 2
    mac_info_df['deauth_total'] = mac_info_df['deauth1_count'] + mac_info_df['deauth2_count'] + mac_info_df['deauth3_count']
    mac_info_df['assoc_succ + auth'] = mac_info_df['assoc_succ_count'] + mac_info_df['auths_count']
    
    return mac_info_df


I think all assoc_successes are duplicated

it's likely that the mac addresses with much more total ( ~ > 1000 ) connections are local pc's/desktops on campus

### Detecting unique mac addresses through code
Need to implement a way to get unique mac addresses throughout the logs instead of just manually selecting for inspection. Regex may be useful for detecting the mac address patterns

In [5]:
import re 

In [9]:
case[0]

'Sep 29 11:24:01 10.44.80.228 stm[2528]:  <501093> <NOTI> |AP EXT-Levi184-AP275-1@10.44.80.228 stm|  Auth success: 38:f9:d3:2e:47:62: AP 10.44.80.228-b4:5d:50:13:45:61-EXT-Levi184-AP275-1\n'

In [10]:
#regex for mac address pattern: https://uibakery.io/regex-library/mac-address
#mac_pattern = '/^(?:[0-9A-Fa-f]{2}[:-]){5}(?:[0-9A-Fa-f]{2}):$/'
pattern = re.compile(r'([0-9a-f]{2}(?::[0-9a-f]{2}){5})', re.IGNORECASE)
mac = re.findall(pattern, case[15])

In [11]:
mac

['38:f9:d3:2e:47:62', '48:4a:e9:d5:ad:d1']

In [12]:
analyze_mac_address(mac)

,mac_address,assoc_req_count,assoc_succ_count,auths_count,deauth1_count,deauth2_count,deauth3_count,deauth_total,assoc_succ + auth
0,38:f9:d3:2e:47:62,40,40.0,41,40,36,2,78,81.0
1,48:4a:e9:d5:ad:d1,879,879.0,931,844,546,129,1519,1810.0


In [14]:
case[15]

'Sep 29 11:25:08 10.47.0.32 stm[1918]: <501080> <1918> <NOTI> < 10.47.0.32>  Deauth to sta: 38:f9:d3:2e:47:62: Ageout AP 10.44.80.205-48:4a:e9:d5:ad:d1-EXT-Levi159-AP375-1 STA has roamed to another AP\n'

### Find all unique Mac Addresses in file

In [6]:
path = '/media/calvinhathcock/Secondary SSD/ITSC_4155_Capstone/dataset/raw/wifi/var/log/remote/wireless-encoded/wireless_09-29-2020.log'
pattern = re.compile(r'([0-9a-f]{2}(?::[0-9a-f]{2}){5})', re.IGNORECASE)
mac_addresses = set()


with open(path, 'r') as file:
    for line in file:
        macs = re.findall(pattern, line)
        for m in macs:
            mac_addresses.add(m)
    

In [10]:
mac_addresses = list(mac_addresses)

In [13]:
new_macs = []
for i in range(0, len(mac_addresses), 500):
    new_macs.append(mac_addresses[i])

In [17]:
new_macs = set(new_macs)

In [ ]:
def check_log_type(line):
    
    code_counts = {
        'assoc_req_count':0, 
        'assoc_succ_count':0,
        'auths_count':0,
        'deauth1_count':0,
        'deauth2_count':0, 
        'deauth3_count':0
    }
    
    if '<501095>' in line:
        code_counts['assoc_req_count'] += 1
    if '<501100>' in line:
        code_counts['assoc_succ_count'] += 1
    if '<501093>' in line:
        code_counts['auths_count'] += 1
    if '<501080>' in line:
        code_counts['deauth1_count'] += 1
    if '<501105>' in line:
        code_counts['deauth2_count'] += 1
    if '<501106>' in line:
        code_counts['deauth1_count'] += 1
        
    return code_counts


ma_df = pd.DataFrame(
    columns = ['assoc_req_count', 'assoc_succ_count', 'auths_count', 'deauth1_count', 'deauth2_count', 'deauth3_count'],
    index = new_macs
).fillna(0)

pattern = re.compile(r'([0-9a-f]{2}(?::[0-9a-f]{2}){5})', re.IGNORECASE)
path = '/media/calvinhathcock/Secondary SSD/ITSC_4155_Capstone/dataset/raw/wifi/var/log/remote/wireless-encoded/wireless_09-29-2020.log'

with open(path, 'r') as file:
    for line in file:
        macs = re.findall(pattern, line)
        code_counts = check_log_type(line)
        for m in macs:
            for code in code_counts.keys():
                try:
                    ma_df.loc[m][code] += code_counts[code]
                except:
                    pass

        
